In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.DotNet.Interactive, 1.0.0-beta.24229.4"


In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

## Adding class with KernelFunction

In [ ]:
class Demographics
{
    [KernelFunction]
    public int GetPersonAge(string name)
    {
        return name switch
        {
            "Marco" => 30,
            _ => 20
        };
    }
}


In [ ]:
Kernel kernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion("gpt-4o", Environment.GetEnvironmentVariable("OAI_KEY")!)
    .Build();

## Add Plugin

In [ ]:
kernel.ImportPluginFromType<Demographics>();
// KernelPlugin plugin = KernelPluginFactory.CreateFromType<Demographics>();
// kernel.Plugins.Add(plugin);

## Allow AI to use 

In [ ]:
var settings = new OpenAIPromptExecutionSettings() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };

In [ ]:
var chatService = kernel.GetRequiredService<IChatCompletionService>();
ChatHistory chat = new();

## Add Settings and Kernel to ChatService

In [ ]:
while (true)
{
    Console.WriteLine("Q: ");
    var message = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Your prompt");
    chat.AddUserMessage(message);
    Console.WriteLine($"User: {message}");
    
    var response = await chatService.GetChatMessageContentAsync(chat, settings, kernel); // <-- Added settings and kernel

    Console.WriteLine(response);
    chat.Add(response);
}